In [16]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point
from keplergl import KeplerGl
import plotly.express as px

In [2]:
# Load OD Data
raw_od_bus = pd.read_csv('od_data/origin_destination_bus_202507.csv')
raw_od_train = pd.read_csv('od_data/origin_destination_train_202507.csv')
combined_od = pd.concat([raw_od_bus, raw_od_train], ignore_index=True)

In [3]:
combined_od

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2025-07,WEEKDAY,6,BUS,71081,1139,3
1,2025-07,WEEKENDS/HOLIDAY,16,BUS,52231,62131,18
2,2025-07,WEEKENDS/HOLIDAY,8,BUS,9212,50229,24
3,2025-07,WEEKENDS/HOLIDAY,18,BUS,9059,42061,2
4,2025-07,WEEKENDS/HOLIDAY,15,BUS,84619,7211,1
...,...,...,...,...,...,...,...
6601961,2025-07,WEEKENDS/HOLIDAY,8,TRAIN,DT31,DT24,43
6601962,2025-07,WEEKENDS/HOLIDAY,15,TRAIN,DT27,EW25,4
6601963,2025-07,WEEKENDS/HOLIDAY,5,TRAIN,NE5,EW2/DT32,1
6601964,2025-07,WEEKDAY,20,TRAIN,BP3,NS11,16


In [4]:
# Extract lat lon from common/non_geo/busstop_current.json

# Read json file 
bus_stops = json.load(open('common/non_geo/busstop_current.json'))
# Convert list of dicts to DataFrame
bus_stops_df = pd.DataFrame(bus_stops)



invalid_bus_stops = [
    ["2051", "Raffles Ave", "The Float @ Marina Bay", 1.289500, 103.859030],
    ["59008", "Yishun Ave 2", "Yishun Int 2", 1.427877, 103.836181]
]

# Convert list of lists to DataFrame
invalid_bus_stops_df = pd.DataFrame(invalid_bus_stops, columns=bus_stops_df.columns)

# Combine with existing DataFrame
bus_stops_df = pd.concat([bus_stops_df, invalid_bus_stops_df], ignore_index=True)

# # Create GeoDataFrame with Point geometry from lon/lat
# bus_stops_gdf = gpd.GeoDataFrame(
#     bus_stops_df,
#     geometry=[Point(xy) for xy in zip(bus_stops_df.Longitude, bus_stops_df.Latitude)],
#     crs="EPSG:4326"  # WGS84 (latitude/longitude)
# )

# print(gdf.head())
# print(gdf.crs)

bus_stops_df["BusStopCode"] = pd.to_numeric(bus_stops_df["BusStopCode"], errors="coerce")

# Merge on bus stop code
combined_od = combined_od.merge(
    bus_stops_df[['BusStopCode', 'Description', 'Latitude', 'Longitude']], 
    left_on='ORIGIN_PT_CODE', right_on='BusStopCode', how='left'
)

combined_od = combined_od.merge(
    bus_stops_df[['BusStopCode', 'Description', 'Latitude', 'Longitude']], 
    left_on='DESTINATION_PT_CODE', right_on='BusStopCode', how='left'
)



In [5]:
bus_stops_df

,BusStopCode,RoadName,Description,Latitude,Longitude
0,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,1029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,1039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...
5159,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5160,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503
5161,99189,Telok Paku Rd,S'pore Aviation Ac,1.388414,103.989716
5162,2051,Raffles Ave,The Float @ Marina Bay,1.289500,103.859030


In [6]:
combined_od
# bus_stops_df

TIME_PER_HOUR = 8
DAY_TYPE = "WEEKDAY"
ORIGIN_PT_CODE = [40389, 40391, 40399, 40401, 40409, 40411, 40419, 40421]
DESTINATION_PT_CODE = [""]


# Existing filters
# mask = (combined_od["TIME_PER_HOUR"] == TIME_PER_HOUR) & (combined_od["DAY_TYPE"] == DAY_TYPE)
mask = (combined_od["DAY_TYPE"] == DAY_TYPE)

# Conditional filter for ORIGIN_PT_CODE
if ORIGIN_PT_CODE:
    mask &= combined_od["ORIGIN_PT_CODE"].isin(ORIGIN_PT_CODE)

# Conditional filter for DESTINATION_PT_CODE
if DESTINATION_PT_CODE and DESTINATION_PT_CODE != [""]:  # ignore empty string list
    mask &= combined_od["DESTINATION_PT_CODE"].isin(DESTINATION_PT_CODE)

# Apply combined mask
filtered_df = combined_od[mask]

# Convert DAY + TIME_PER_HOUR into a proper timestamp
filtered_df["Timestamp"] = (pd.to_datetime("1990-01-01") + 
                      pd.to_timedelta(filtered_df["TIME_PER_HOUR"], unit='h')).dt.strftime('%Y-%m-%dT%H:%M:%S')

C:\Users\renji\AppData\Local\Temp\ipykernel_27008\3318732272.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Timestamp"] = (pd.to_datetime("1990-01-01") +


In [7]:
filtered_df

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,BusStopCode_x,Description_x,Latitude_x,Longitude_x,BusStopCode_y,Description_y,Latitude_y,Longitude_y,Timestamp
2367,2025-07,WEEKDAY,10,BUS,40401,43189,10,40401.0,Blk 224A,1.357260,103.731210,43189.0,Blk 283,1.346993,103.756815,1990-01-01T10:00:00
15695,2025-07,WEEKDAY,6,BUS,40411,40311,32,40411.0,Blk 221B,1.357550,103.729300,40311.0,Blk 439C,1.357305,103.738908,1990-01-01T06:00:00
17158,2025-07,WEEKDAY,19,BUS,40411,43751,344,40411.0,Blk 221B,1.357550,103.729300,43751.0,Opp Blk 443D,1.356298,103.736664,1990-01-01T19:00:00
26954,2025-07,WEEKDAY,5,BUS,40401,43009,476,40401.0,Blk 224A,1.357260,103.731210,43009.0,Bt Batok Int,1.349994,103.751062,1990-01-01T05:00:00
34267,2025-07,WEEKDAY,6,BUS,40401,43601,135,40401.0,Blk 224A,1.357260,103.731210,43601.0,Opp Blk 532,1.356643,103.749505,1990-01-01T06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5713377,2025-07,WEEKDAY,11,BUS,40421,42079,4,40421.0,Blk 220A,1.359380,103.729694,42079.0,Opp Pei Hwa Presby Pr Sch,1.339255,103.776989,1990-01-01T11:00:00
5714662,2025-07,WEEKDAY,14,BUS,40401,43189,2,40401.0,Blk 224A,1.357260,103.731210,43189.0,Blk 283,1.346993,103.756815,1990-01-01T14:00:00
5717112,2025-07,WEEKDAY,19,BUS,40409,40429,2,40409.0,Blk 132B,1.357394,103.731808,40429.0,Blk 242,1.359400,103.729300,1990-01-01T19:00:00
5717450,2025-07,WEEKDAY,17,BUS,40389,42161,1,40389.0,Tengah CC,1.356473,103.734424,42161.0,Blk 18,1.340277,103.772646,1990-01-01T17:00:00


In [8]:
from shapely.geometry import LineString
import geopandas as gpd

# Create LineString geometry from origin to destination
filtered_df["geometry"] = filtered_df.apply(
    lambda row: LineString([(row.Latitude_x, row.Longitude_x), (row.Latitude_y, row.Longitude_y)]),
    axis=1
)

gdf_od = gpd.GeoDataFrame(filtered_df, geometry="geometry", crs="EPSG:4326")

C:\Users\renji\AppData\Local\Temp\ipykernel_27008\2102711312.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["geometry"] = filtered_df.apply(


In [9]:
config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [
                {
                    "id": "timestamp_filter",
                    "dataId": ["OD_Trips"],
                    "name": ["Timestamp"],   # column in your df
                    "type": "timeRange",
                    "enabled": True,
                    "value": [
                        "1990-01-01T08:00:00",   # start time (can be earliest in your data)
                        "1990-01-01T09:00:00"    # end time (1h later)
                    ],                    
                    "enlarged": True,
                    "plotType": "histogram",
                    "animationWindow": "incremental",
                    "speed": 1
                }
            ],
            "layers": [
                {
                    "id": "od_layer",
                    "type": "arc",
                    "config": {
                        "dataId": "OD_Trips",
                        "label": "OD Arcs",
                        "color": [255, 0, 0],
                        "columns": {
                            "lat0": "Latitude_x",
                            "lng0": "Longitude_x",
                            "lat1": "Latitude_y",
                            "lng1": "Longitude_y"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "thickness": 2,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            }
                        }
                    }
                }
            ],
            "interactionConfig": {},
            "layerBlending": "normal",
            "splitMaps": []
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
            "latitude": 1.3728880599008766,
            "longitude": 103.80202140824348,
            "pitch": 0,
            "zoom": 14,
            "isSplit": False
        },
        # "mapStyle": {
        #     "styleType": "dark",
        #     "topLayerGroups": {},
        #     "visibleLayerGroups": {}
        # }
    }
}


In [10]:
map_1 = KeplerGl(config=config)

df_clean = gdf_od.dropna(subset=["BusStopCode_x"])

# Add dataframe as a dataset
map_1.add_data(data=df_clean, name="OD_Trips")



User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [11]:
df_clean.loc[df_clean["BusStopCode_y"].isna(), "DESTINATION_PT_CODE"].unique()

array([], dtype=object)

In [12]:
df_clean[df_clean["BusStopCode_y"].isna()]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,BusStopCode_x,Description_x,Latitude_x,Longitude_x,BusStopCode_y,Description_y,Latitude_y,Longitude_y,Timestamp,geometry


In [13]:
df_clean

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,BusStopCode_x,Description_x,Latitude_x,Longitude_x,BusStopCode_y,Description_y,Latitude_y,Longitude_y,Timestamp,geometry
2367,2025-07,WEEKDAY,10,BUS,40401,43189,10,40401.0,Blk 224A,1.357260,103.731210,43189.0,Blk 283,1.346993,103.756815,1990-01-01T10:00:00,"LINESTRING (1.35726 103.73121, 1.34699 103.75682)"
15695,2025-07,WEEKDAY,6,BUS,40411,40311,32,40411.0,Blk 221B,1.357550,103.729300,40311.0,Blk 439C,1.357305,103.738908,1990-01-01T06:00:00,"LINESTRING (1.35755 103.7293, 1.3573 103.73891)"
17158,2025-07,WEEKDAY,19,BUS,40411,43751,344,40411.0,Blk 221B,1.357550,103.729300,43751.0,Opp Blk 443D,1.356298,103.736664,1990-01-01T19:00:00,"LINESTRING (1.35755 103.7293, 1.3563 103.73666)"
26954,2025-07,WEEKDAY,5,BUS,40401,43009,476,40401.0,Blk 224A,1.357260,103.731210,43009.0,Bt Batok Int,1.349994,103.751062,1990-01-01T05:00:00,"LINESTRING (1.35726 103.73121, 1.34999 103.75106)"
34267,2025-07,WEEKDAY,6,BUS,40401,43601,135,40401.0,Blk 224A,1.357260,103.731210,43601.0,Opp Blk 532,1.356643,103.749505,1990-01-01T06:00:00,"LINESTRING (1.35726 103.73121, 1.35664 103.74951)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5713377,2025-07,WEEKDAY,11,BUS,40421,42079,4,40421.0,Blk 220A,1.359380,103.729694,42079.0,Opp Pei Hwa Presby Pr Sch,1.339255,103.776989,1990-01-01T11:00:00,"LINESTRING (1.35938 103.72969, 1.33926 103.77699)"
5714662,2025-07,WEEKDAY,14,BUS,40401,43189,2,40401.0,Blk 224A,1.357260,103.731210,43189.0,Blk 283,1.346993,103.756815,1990-01-01T14:00:00,"LINESTRING (1.35726 103.73121, 1.34699 103.75682)"
5717112,2025-07,WEEKDAY,19,BUS,40409,40429,2,40409.0,Blk 132B,1.357394,103.731808,40429.0,Blk 242,1.359400,103.729300,1990-01-01T19:00:00,"LINESTRING (1.35739 103.73181, 1.3594 103.7293)"
5717450,2025-07,WEEKDAY,17,BUS,40389,42161,1,40389.0,Tengah CC,1.356473,103.734424,42161.0,Blk 18,1.340277,103.772646,1990-01-01T17:00:00,"LINESTRING (1.35647 103.73442, 1.34028 103.77265)"


In [14]:
from keplergl import KeplerGl

def save_fullwidth_html(map_obj, file_name="map.html", height="100vh"):
    """
    Save a KeplerGl map with width=100% and full viewport height by default.
    """
    # First save normally
    map_obj.save_to_html(file_name=file_name)

    # Now patch inline style in HTML
    with open(file_name, "r", encoding="utf-8") as f:
        html = f.read()

    # Replace the container div inline style
    html = html.replace(
        'style="width: 1000px; height: 500px;"',
        f'style="width: 100%; height: {height};"'
    )

    with open(file_name, "w", encoding="utf-8") as f:
        f.write(html)

    print(f"✔ Map saved to {file_name} with full-width layout")

save_fullwidth_html(map_1, "od_map.html", height="100vh")

Map saved to od_map.html!
✔ Map saved to od_map.html with full-width layout


In [30]:
# Create bar chart
fig = px.bar(df_clean[df_clean["ORIGIN_PT_CODE"] == 40401], x='Timestamp', y='TOTAL_TRIPS', title='Bar Chart Example')

# Show figure
fig.show()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,BusStopCode_x,Description_x,Latitude_x,Longitude_x,BusStopCode_y,Description_y,Latitude_y,Longitude_y,Timestamp,geometry
2367,2025-07,WEEKDAY,10,BUS,40401,43189,10,40401.0,Blk 224A,1.35726,103.73121,43189.0,Blk 283,1.346993,103.756815,1990-01-01T10:00:00,"LINESTRING (1.35726 103.73121, 1.34699 103.75682)"
26954,2025-07,WEEKDAY,5,BUS,40401,43009,476,40401.0,Blk 224A,1.35726,103.73121,43009.0,Bt Batok Int,1.349994,103.751062,1990-01-01T05:00:00,"LINESTRING (1.35726 103.73121, 1.34999 103.75106)"
34267,2025-07,WEEKDAY,6,BUS,40401,43601,135,40401.0,Blk 224A,1.35726,103.73121,43601.0,Opp Blk 532,1.356643,103.749505,1990-01-01T06:00:00,"LINESTRING (1.35726 103.73121, 1.35664 103.74951)"
47737,2025-07,WEEKDAY,5,BUS,40401,42159,60,40401.0,Blk 224A,1.35726,103.73121,42159.0,Opp Beauty World Stn,1.340368,103.775309,1990-01-01T05:00:00,"LINESTRING (1.35726 103.73121, 1.34037 103.77531)"
52078,2025-07,WEEKDAY,13,BUS,40401,43179,2,40401.0,Blk 224A,1.35726,103.73121,43179.0,Blk 231,1.348971,103.754303,1990-01-01T13:00:00,"LINESTRING (1.35726 103.73121, 1.34897 103.7543)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5601669,2025-07,WEEKDAY,12,BUS,40401,40311,6,40401.0,Blk 224A,1.35726,103.73121,40311.0,Blk 439C,1.357305,103.738908,1990-01-01T12:00:00,"LINESTRING (1.35726 103.73121, 1.3573 103.73891)"
5639785,2025-07,WEEKDAY,16,BUS,40401,43409,34,40401.0,Blk 224A,1.35726,103.73121,43409.0,Opp Blk 127,1.352355,103.746236,1990-01-01T16:00:00,"LINESTRING (1.35726 103.73121, 1.35236 103.74624)"
5661547,2025-07,WEEKDAY,5,BUS,40401,43549,1,40401.0,Blk 224A,1.35726,103.73121,43549.0,Regent Hts,1.352775,103.755295,1990-01-01T05:00:00,"LINESTRING (1.35726 103.73121, 1.35278 103.7553)"
5683304,2025-07,WEEKDAY,20,BUS,40401,43751,61,40401.0,Blk 224A,1.35726,103.73121,43751.0,Opp Blk 443D,1.356298,103.736664,1990-01-01T20:00:00,"LINESTRING (1.35726 103.73121, 1.3563 103.73666)"
